In [8]:
import pandas as pd

In [10]:
file_path = "Downloads/CLOUDTHAT_FOLDER/Vehicle_Sales_2024_CSV.csv"
df = pd.read_csv(file_path)
df.head()

,slno,model_desc,fuel,colour,vehicle_class,make_year,seat_capacity,second_vehicle,temp_registration_number,category,maker_name,office_cd,from_date,vehicle_age,group
0,1,MARUTI TOUR M(O) CNG 1.5L 5MT BSVI-PH2,CNG PETROL,PEARL ARCTIC WHITE,MOTOR CAB,01-12-2023,7,N,TS24NTR9216,Transport,"MARUTI SUZUKI INDIA LTD.,",RTA KHAMMAM,01-01-2024,1,Traditional
1,2,TVS XL100 HEAVY DUTY I-TOUCH START BSVI-PH2,PETROL,DBLU,MOTOR CYCLE,01-10-2023,2,N,TS02BJTR1342,Non Transport,TVS MOTOR COMPANY LTD,UNIT OFFICE RAMAGUNDAM,01-01-2024,1,Traditional
2,3,ACTIVA 125 DISC BSVI-PH2,PETROL,PEARL NIGHTSTAR BLAC,MOTOR CYCLE,01-11-2023,2,N,TS04APTR5126,Non Transport,HONDA MOTORCYCLE&SCOOTER(I)P L,RTA KHAMMAM,01-01-2024,1,Traditional
3,4,TVS ZEST BSVI-PH2,PETROL,MATTE BLACK,MOTOR CYCLE,01-10-2023,2,N,TS09GMTR6782,Non Transport,TVS MOTOR COMPANY LTD,RTA UPPAL,01-01-2024,1,Traditional
4,5,LS1510.3T6RB 5200 WB 40+D BSVI-PH2,DIESEL,WHITE,OMINIBUS,01-08-2023,41,N,TS10CCTR7431,Transport,ASHOK LEYLAND LTD,RTA-HYDERABAD-EZ,01-01-2024,1,Traditional


In [12]:
# Parse month from from_date (assume format YYYY-MM-DD)
df['fromdate'] = pd.to_datetime(df['from_date'], errors='coerce')
df['month'] = df['fromdate'].dt.month
df['year'] = df['fromdate'].dt.year

In [14]:
# Aggregate registration count per group (one row = one combination per month)
grouped = (
    df.groupby(['vehicle_class', 'model_desc', 'office_cd', 'month'])
      .size()
      .reset_index(name='registration_count')
)

In [16]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
for col in ['vehicle_class', 'model_desc', 'office_cd']:
    le = LabelEncoder()
    grouped[col] = le.fit_transform(grouped[col])

In [18]:
from sklearn.model_selection import train_test_split

X = grouped[['vehicle_class', 'model_desc', 'office_cd', 'month']]
y = grouped['registration_count']

In [20]:
X.shape

(114220, 4)

In [22]:
y.shape

(114220,)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Train the CART model
cart_model = DecisionTreeRegressor(random_state=42)
cart_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [28]:
# Predict and evaluate
y_pred_cart = cart_model.predict(X_test)
print("Decision Tree Test MSE:", mean_squared_error(y_test, y_pred_cart))
print("Decision Tree R2 Score:", r2_score(y_test, y_pred_cart))

Decision Tree Test MSE: 10.97220276659079
Decision Tree R2 Score: 0.854010308804737


In [30]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [32]:
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Test MSE:", mean_squared_error(y_test, y_pred_rf))
print("Random Forest R2 Score:", r2_score(y_test, y_pred_rf))

Random Forest Test MSE: 7.977176142531956
Random Forest R2 Score: 0.8938603754931963


# Prediction of Registrations

In [34]:
X_new = pd.DataFrame({
    'vehicle_class': [14],  # MOTOR CYCLE
    'model_desc': [3217],    # ACTIVA-6G LED BSVI - 291; OLA S1 PRO (GEN2) BOV: 3217
    'office_cd': [25],      # RTA RANGAREDDY
    'month': [10],          # October
})

# Predict with CART
print('CART Prediction:', cart_model.predict(X_new))
# Predict with Random Forest
print('Random Forest Prediction:', rf_model.predict(X_new))

CART Prediction: [1.]
Random Forest Prediction: [2.87]


In [ ]:
# To get code for vehicle class

In [36]:
from sklearn.preprocessing import LabelEncoder

# vehicle_class encoding
le_vehicle_class = LabelEncoder().fit(df['vehicle_class'])
vehicle_class_mapping = dict(zip(le_vehicle_class.classes_, range(len(le_vehicle_class.classes_))))

vehicle_class_mapping = dict(zip(le_vehicle_class.classes_, range(len(le_vehicle_class.classes_))))
print("Vehicle class to code mapping:")
for cls, code in vehicle_class_mapping.items():
    print(f"{cls}: {code}")

Vehicle class to code mapping:
AMBULANCE: 0
ARTICULATED VEHICLES: 1
AUTO RICKSHAW: 2
AUTO RIKCKSHAW PRIVATE: 3
CHASSIS TRANSPORT: 4
CRANE FOR PRIVATE USE: 5
ECART: 6
ERICKSHAW: 7
FORK LIFT: 8
GOODS CARRIAGE: 9
LOADER: 10
MAXI CAB: 11
MOTOR CAB: 12
MOTOR CAR: 13
MOTOR CYCLE: 14
MOTOR CYCLE FOR HIRE: 15
MOTOR GRADER: 16
OMINIBUS: 17
OMNIBUS FOR PRIVATE USE: 18
ROAD ROLLER: 19
SELF LOADING CONCRETE MIXER: 20
SELF PROPELLED HARVESTER: 21
THREE WHEELED GOODS VEHICLE: 22
TRACTOR DRIVEN COMBINED HARVESTER: 23
TRACTOR FOR AGRICULTURAL PURPOSE: 24
TRACTOR FOR COMMERCIAL USE: 25
TRAILER FOR AGRICULTURE PURPOSE: 26
TRAILER FOR COMMERCIAL USE: 27
VEHICLE FITTED WITH CONSTRUCTION EQUIPMENT: 28


In [38]:
vehicle_name = 'MOTOR CYCLE'
code = le_vehicle_class.transform([vehicle_name])[0]
print(f"The code for {vehicle_name} is: {code}")

The code for MOTOR CYCLE is: 14


In [ ]:
# To get code for model

In [40]:
le_model_desc = LabelEncoder()
le_model_desc.fit(df['model_desc'])

# Now you can get the mapping from model name to assigned code
model_desc_mapping = dict(zip(le_model_desc.classes_, range(len(le_model_desc.classes_))))
print("Model Desc to code mapping:")
for model, code in model_desc_mapping.items():
    print(f"{model}: {code}")

Model Desc to code mapping:
 WATER TANKER  5TONS  2WHEELER TRAILER: 0
10 TON 4WHEEL SINGLE AXLE SEMI TRAILER: 1
10 TONNS 4 WH TRACTOR TRAILER: 2
10 TONS 4 WHEELER SINGLE AXLE FLAT BED: 3
10 TONS FOUR WHEELER TRACTOR  TRAILOR: 4
1012 LPT DCR 35 CBC 125B6M5 BSVI-PH2: 5
1012 LPT DCR 35 HSD 125B6M5 BSVI-PH2: 6
1012 LPT DCR 39 CBC 125B6M5 BSVI-PH2: 7
1012 LPT DCR 39HSD 125B6M5 BSVI-PH2: 8
1012 LPT DCR 45 HSD 125B6M5 BSVI-PH2: 9
10TON 4WHEELERSINGLEAXLESEMITIPPINGTRAILER: 10
10TONS 4 WHEELER SINGLE AXLE SEMI TTIOB BSIV: 11
10TONS 4 WHEELER SINGLE AXLE SEMI TTRAILR: 12
10TONS 4 WHEELER SINGLE AXLE TRAILER: 13
10TONS 4WH SEMI TRAILER IRON BODY: 14
10TONS 4WHEELER SEMISINGLE AXLE FLATBED OP: 15
10TONS 4WHEELER SINGLE AXLE SEMI-TRAILER: 16
10TONS4WHEELER SINGLEAXLEEMITRAILERIRONBOY: 17
1109G LPT DCR 39 HSD 85B6M5 BSVI-PH2: 18
1112 LPT DCR 35 CBC 125B6M5 BSVI-PH2: 19
1112 LPT DCR 39 CBC 125B6M5 BSVI-PH2: 20
1112 LPT DCR 39 HSD 125B6M5 BSVI-PH2: 21
1112 LPT DCR 45 CBC 125B6M5 BSVI-PH2: 22
1112 LPT

In [42]:
model_name = 'ACTIVA-6G LED BSVI' #ACTIVA-6G LED BSVI ; OLA S1 PRO (GEN2) BOV: 3217
code = le_model_desc.transform([model_name])[0]
print(f"The code for {model_name} is: {code}")

The code for ACTIVA-6G LED BSVI is: 291


In [ ]:
# To get code for office

In [44]:
le_office_cd = LabelEncoder().fit(df['office_cd'])
office_cd_mapping = dict(zip(le_office_cd.classes_, range(len(le_office_cd.classes_))))

In [46]:
office_cd_mapping = dict(zip(le_office_cd.classes_, range(len(le_office_cd.classes_))))
print("Office code mapping:")
for office, code in office_cd_mapping.items():
    print(f"{office}: {code}")


Office code mapping:
RTA ADILABAD: 0
RTA BHADRADRI: 1
RTA HANUMAKONDA: 2
RTA IBRAHIMPATNAM: 3
RTA JAGITYAL: 4
RTA JANGOAN: 5
RTA JAYASHANKAR: 6
RTA JOGULAMBA: 7
RTA KAMAREDDY: 8
RTA KARIMNAGAR: 9
RTA KHAMMAM: 10
RTA KOMRAMBHEEM: 11
RTA MAHABOOBNAGAR: 12
RTA MAHABUBABAD: 13
RTA MANCHERIAL: 14
RTA MEDAK: 15
RTA MEDCHAL: 16
RTA MULUGU: 17
RTA NAGARKURNOOL: 18
RTA NALGONDA: 19
RTA NARAYANPET: 20
RTA NIRMAL: 21
RTA NIZAMABAD: 22
RTA PEDDAPALLI: 23
RTA RAJANNA: 24
RTA RANGAREDDY: 25
RTA SANGAREDDY: 26
RTA SIDDIPET: 27
RTA SURYAPET: 28
RTA UPPAL: 29
RTA VIKARABAD: 30
RTA WANAPARTHY: 31
RTA WARANGAL: 32
RTA YADADRI: 33
RTA-HYDERABAD-CZ: 34
RTA-HYDERABAD-EZ: 35
RTA-HYDERABAD-NZ: 36
RTA-HYDERABAD-SZ: 37
RTA-HYDERABAD-WZ: 38
UNIT OFFICE ARMOOR: 39
UNIT OFFICE BHADRACHALAM: 40
UNIT OFFICE BHODAN: 41
UNIT OFFICE HUZURABAD: 42
UNIT OFFICE KALWAKURTHY: 43
UNIT OFFICE KODAD: 44
UNIT OFFICE KORUTLA: 45
UNIT OFFICE KUKATPALLY: 46
UNIT OFFICE MIRYALAGUDA: 47
UNIT OFFICE PARGI: 48
UNIT OFFICE PATANCHERUVU

In [48]:
office_name = 'RTA RANGAREDDY'
code = le_office_cd.transform([office_name])[0]
print(f"The code for {office_name} is: {code}")

The code for RTA RANGAREDDY is: 25


## Pattern matching

In [50]:
import re

def get_matching_classes(pattern, le):
    regex_pattern = '^' + pattern.replace('%', '.*') + '$'

    matching = []
    for cls in le.classes_:
        if re.match(regex_pattern, cls, re.IGNORECASE):
            code = le.transform([cls])[0]
            matching.append((cls, code))
    return matching

# vehicle_class:
pattern = 'MOTOR%'
matches = get_matching_classes(pattern, le_vehicle_class)

if matches:
    print(f"Vehicle classes matching pattern '{pattern}':")
    for name, code in matches:
        print(f"{name}: {code}")
else:
    print(f"No vehicle classes match the pattern '{pattern}'")

Vehicle classes matching pattern 'MOTOR%':
MOTOR CAB: 12
MOTOR CAR: 13
MOTOR CYCLE: 14
MOTOR CYCLE FOR HIRE: 15
MOTOR GRADER: 16


In [52]:
# For model_desc
pattern_model = 'ACTIVA%'  # Example pattern: all models starting with 'ACTIVA'
matches_model = get_matching_classes(pattern_model, le_model_desc)

if matches_model:
    print(f"Model descriptions matching pattern '{pattern_model}':")
    for name, code in matches_model:
        print(f"{name}: {code}")
else:
    print(f"No model descriptions match the pattern '{pattern_model}'")

Model descriptions matching pattern 'ACTIVA%':
ACTIVA - 6G DLX BSVI: 278
ACTIVA - 6G STD BSVI: 279
ACTIVA 125 DISC BSVI-PH2: 280
ACTIVA 125 DISC. BSVI-PH2: 281
ACTIVA 125 DRUM ALLOY BSVI-PH2: 282
ACTIVA 125 DRUM BSVI-PH2: 283
ACTIVA 125 SMART BSVI-PH2: 284
ACTIVA 125 SMART. BSVI-PH2: 285
ACTIVA 6G WITH LED HEAD LIGHT BSVI: 286
ACTIVA 6G WITH SHEET WHEEL BSVI: 287
ACTIVA- DLX BSVI: 288
ACTIVA-125- DISC BSVI: 289
ACTIVA-125- DRUM BSVI: 290
ACTIVA-6G LED BSVI: 291
ACTIVA-DLX BSVI-PH2: 292
ACTIVA-SMART BSVI-PH2: 293
ACTIVA-STD BSVI-PH2: 294


In [54]:
# For office_cd
pattern_office = 'RTA%'  # Example pattern: all office codes starting with 'RTA'
matches_office = get_matching_classes(pattern_office, le_office_cd)

if matches_office:
    print(f"Office codes matching pattern '{pattern_office}':")
    for name, code in matches_office:
        print(f"{name}: {code}")
else:
    print(f"No office codes match the pattern '{pattern_office}'")

Office codes matching pattern 'RTA%':
RTA ADILABAD: 0
RTA BHADRADRI: 1
RTA HANUMAKONDA: 2
RTA IBRAHIMPATNAM: 3
RTA JAGITYAL: 4
RTA JANGOAN: 5
RTA JAYASHANKAR: 6
RTA JOGULAMBA: 7
RTA KAMAREDDY: 8
RTA KARIMNAGAR: 9
RTA KHAMMAM: 10
RTA KOMRAMBHEEM: 11
RTA MAHABOOBNAGAR: 12
RTA MAHABUBABAD: 13
RTA MANCHERIAL: 14
RTA MEDAK: 15
RTA MEDCHAL: 16
RTA MULUGU: 17
RTA NAGARKURNOOL: 18
RTA NALGONDA: 19
RTA NARAYANPET: 20
RTA NIRMAL: 21
RTA NIZAMABAD: 22
RTA PEDDAPALLI: 23
RTA RAJANNA: 24
RTA RANGAREDDY: 25
RTA SANGAREDDY: 26
RTA SIDDIPET: 27
RTA SURYAPET: 28
RTA UPPAL: 29
RTA VIKARABAD: 30
RTA WANAPARTHY: 31
RTA WARANGAL: 32
RTA YADADRI: 33
RTA-HYDERABAD-CZ: 34
RTA-HYDERABAD-EZ: 35
RTA-HYDERABAD-NZ: 36
RTA-HYDERABAD-SZ: 37
RTA-HYDERABAD-WZ: 38


# Prediction of Registrations using all elements

In [56]:
# All encoded values for each category
all_vehicle_classes = le_vehicle_class.transform(le_vehicle_class.classes_)
all_model_descs    = le_model_desc.transform(le_model_desc.classes_)
all_office_cds     = le_office_cd.transform(le_office_cd.classes_)
all_months         = list(range(1, 13))  # Months 1 to 12

In [58]:
# INPUT THE VALUES FOR PREDICTION

V = len(all_vehicle_classes)
D = len(all_model_descs)
O = len(all_office_cds)
M = len(all_months)

X_new = pd.DataFrame({
    'vehicle_class': all_vehicle_classes,
    'model_desc': [284] * V,      # Repeat N times
    'office_cd': [5] * V,        # Repeat N times
    'month': [7] * V             # Repeat N times
})

# Predict with CART
print('CART Prediction:', cart_model.predict(X_new))
# Predict with Random Forest
print('Random Forest Prediction:', rf_model.predict(X_new))

CART Prediction: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 8. 8. 8. 8. 8. 8. 8. 8.
 1. 1. 1. 1. 1.]
Random Forest Prediction: [1.08 1.08 1.08 1.16 1.22 1.11 1.11 1.07 1.07 1.07 1.04 1.02 1.02 1.
 1.34 2.53 2.51 2.51 2.51 2.51 2.51 2.51 2.51 2.51 2.02 2.02 2.02 2.02
 2.02]


In [60]:
# TABLE FORMAT

features = ['vehicle_class', 'model_desc', 'office_cd', 'month']
cart_preds = cart_model.predict(X_new[features])
rf_preds = rf_model.predict(X_new[features])

# Decode codes back to names for readability
vehicle_names = le_vehicle_class.inverse_transform(X_new['vehicle_class'])
model_names = le_model_desc.inverse_transform(X_new['model_desc'])
office_names = le_office_cd.inverse_transform(X_new['office_cd'])

# Build results DataFrame
results_df = pd.DataFrame({
    'vehicle_class': vehicle_names,
    'model_desc': model_names,
    'office_cd': office_names,
    'CART_Prediction': cart_preds,
    'RF_Prediction': rf_preds
})

results_df.head(10)

,vehicle_class,model_desc,office_cd,CART_Prediction,RF_Prediction
0,AMBULANCE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.08
1,ARTICULATED VEHICLES,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.08
2,AUTO RICKSHAW,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.08
3,AUTO RIKCKSHAW PRIVATE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.16
4,CHASSIS TRANSPORT,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.22
5,CRANE FOR PRIVATE USE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.11
6,ECART,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.11
7,ERICKSHAW,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.07
8,FORK LIFT,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.07
9,GOODS CARRIAGE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,1.0,1.07


# Selecting 'all' as values for more than one feature

In [62]:
import itertools
combos = list(itertools.product(all_vehicle_classes, [15], all_office_cds, [7]))
X_new = pd.DataFrame(combos, columns=['vehicle_class', 'model_desc', 'office_cd', 'month'])

In [74]:
# Check

combos = list(itertools.product(all_vehicle_classes, [284], [5], [7]))
X_new = pd.DataFrame(combos, columns=['vehicle_class', 'model_desc', 'office_cd', 'month'])

In [76]:
X_new['CART Prediction'] = cart_model.predict(X_new[['vehicle_class', 'model_desc', 'office_cd', 'month']])
X_new['Random Forest Prediction'] = rf_model.predict(X_new[['vehicle_class', 'model_desc', 'office_cd', 'month']])

# Decode category codes to names for clarity
X_new['vehicle_class_name'] = le_vehicle_class.inverse_transform(X_new['vehicle_class'])
X_new['model_desc_name'] = le_model_desc.inverse_transform(X_new['model_desc'])
X_new['office_cd_name'] = le_office_cd.inverse_transform(X_new['office_cd'])

In [78]:
# Preview results

X_new[['vehicle_class_name', 'model_desc_name', 'office_cd_name', 'month', 'CART Prediction', 'Random Forest Prediction']]

,vehicle_class_name,model_desc_name,office_cd_name,month,CART Prediction,Random Forest Prediction
0,AMBULANCE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.08
1,ARTICULATED VEHICLES,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.08
2,AUTO RICKSHAW,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.08
3,AUTO RIKCKSHAW PRIVATE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.16
4,CHASSIS TRANSPORT,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.22
5,CRANE FOR PRIVATE USE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.11
6,ECART,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.11
7,ERICKSHAW,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.07
8,FORK LIFT,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.07
9,GOODS CARRIAGE,ACTIVA 125 SMART BSVI-PH2,RTA JANGOAN,7,1.0,1.07


In [ ]:
# Export as csv file
X_new.to_csv("C:/Users/conne/Downloads/predictions_all_combos.csv", index=False)